# First Exploration of LSST COMCAM

- author : Sylvie Dagoret-Campagne
- creation date :  2025-02-19
- update : 2025-02-19
- w_2025_04

  
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399


``Shemas``:
https://dm.lsst.org/sdm_schemas/browser/

In [ ]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst
from lsst.geom import Angle 

In [ ]:
from collections import OrderedDict

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
# Astropy
from astropy.visualization import ZScaleInterval, AsinhStretch

import seaborn as sns
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd


#xx-small
#x-small
#small
#medium
#large
#x-large
#xx-large

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates
import astropy.coordinates as coord
import astropy.units as u

## Start initialisation here

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")
NDET = 9

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

In [ ]:
# List a number of usefull data-product to explore
if 0:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name) )
            ):
                print(datasetType)

## How do I figure out which tracts have data?

In [ ]:
for dtype in sorted(registry.queryDatasetTypes(expression="*nImage*")):
    print(dtype.name)

In [ ]:
#nImage_refs = list(butler.registry.queryDatasets('deepCoadd_nImage'))

In [ ]:
#tracts = np.unique([ref.dataId['tract'] for ref in nImage_refs])
#print(tracts)
#
#bands = np.unique([ref.dataId['band'] for ref in nImage_refs])
#print(bands)

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)

In [ ]:
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

In [ ]:
dict_tract_to_coords_deg = OrderedDict()
dict_tract_to_coords_rad = OrderedDict()

for tract_sel in tractsId_list:
    the_tract = skymap.generateTract(tract_sel)
    sp2 = the_tract.getCtrCoord() # get a SpherePoint(278.7804878048781*degrees, -48.34710743801652*degrees)
    dict_tract_to_coords_deg[tract_sel] = sp2
    dict_tract_to_coords_rad[tract_sel] = dict(ra = sp2[0].asRadians(), dec = sp2[1].asRadians() )

In [ ]:
#for key, value in dict_tract_To_Coords.items():
#    print(key, "->", value,  " <=> (", value[0].asRadians()," , ", value[1].asRadians(), ")")

In [ ]:
dict_tract_to_coords_deg 

In [ ]:
dict_tract_to_coords_rad

In [ ]:
# Check which tracts actually have a lot of visit coverage:
dict_tract_to_coords_andvisits = OrderedDict()
for tract_sel in tractsId_list:
    visits = list(butler.registry.queryDatasets('visitSummary', tract=tract_sel, skymap=skymapName, findFirst=True))
    Nvis = len(visits)
    dict_tract_to_coords_andvisits[tract_sel] = dict(ra_deg=dict_tract_to_coords_deg[tract_sel][0], dec_deg =dict_tract_to_coords_deg[tract_sel][1] , 
                                                     ra_rad = dict_tract_to_coords_rad[tract_sel]["ra"], dec_rad = dict_tract_to_coords_rad[tract_sel]["dec"],nvis=Nvis)
    

In [ ]:
dict_tract_to_coords_andvisits

## visitSummary

In [ ]:
if 0:

    all_data = []
    data_product = 'visitSummary'

    #datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= "instrument='LSSTComCamSim'")
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection, where = where_clause)
    for i, ref in enumerate(datasetRefs):
        if (i % 100 == 0):
            print(f"======================== datasetType = {data_product} ============================================")
            print("fullId..................:",ref.dataId.full)
        try:
            data = butler.get(data_product, dataId=ref.dataId )  
            all_data.append(data)
       
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)         

    N = len(all_data)
    print(f"{visitSummary} :: N = {N}")

### CCD VisitTable

In [ ]:
list(butler.registry.queryDatasets('ccdVisitTable'))

In [ ]:
ccdVisitTable = butler.get('ccdVisitTable')

In [ ]:
ccdVisitTable.columns

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x : 1./np.cos(func_degToRad(x))
#func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

In [ ]:
ccdVisitTable["airmass"] = ccdVisitTable['zenithDistance'].apply(func_zendtoAirmass)

In [ ]:
f2c = {'u': 'blue', 'g': 'green', 'r': 'red',
       'i': 'orange', 'z': 'grey', 'y': 'k'}

plt.figure(dpi=200)
for bandname in f2c:
    in_band = ccdVisitTable['band'] == bandname
    if np.sum(in_band) > 0:
        plt.plot(ccdVisitTable['zenithDistance'][in_band], ccdVisitTable['zeroPoint'][in_band], 
                 'o', markersize=1, color=f2c[bandname], label=bandname)
plt.legend()
plt.xlabel('zenithDistance')
plt.ylabel('zeroPoint')

In [ ]:
f2c = {'u': 'blue', 'g': 'green', 'r': 'red',
       'i': 'orange', 'z': 'grey', 'y': 'k'}

plt.figure(dpi=200)
for bandname in f2c:
    in_band = ccdVisitTable['band'] == bandname
    if np.sum(in_band) > 0:
        plt.plot(ccdVisitTable['airmass'][in_band], ccdVisitTable['zeroPoint'][in_band], 
                 'o', markersize=1, color=f2c[bandname], label=bandname)
plt.legend()
plt.xlabel('airmass')
plt.ylabel('zeroPoint')

In [ ]:
f2c = {'u': 'blue', 'g': 'green', 'r': 'red',
       'i': 'orange', 'z': 'grey', 'y': 'k'}

plt.figure(dpi=200)
for bandname in f2c:
    in_band = ccdVisitTable['band'] == bandname
    if np.sum(in_band) > 0:
        plt.plot(ccdVisitTable['airmass'][in_band], ccdVisitTable['psfSigma'][in_band], 
                 'o', markersize=1, color=f2c[bandname], label=bandname)
plt.legend()
plt.xlabel('airmass')
plt.ylabel('psfSigma')

In [ ]:
f2c = {'u': 'blue', 'g': 'green', 'r': 'red',
       'i': 'orange', 'z': 'grey', 'y': 'k'}

plt.figure(dpi=200)
for bandname in f2c:
    in_band = ccdVisitTable['band'] == bandname
    if np.sum(in_band) > 0:
        plt.plot(ccdVisitTable['airmass'][in_band], ccdVisitTable['seeing'][in_band], 
                 'o', markersize=1, color=f2c[bandname], label=bandname)
plt.legend()
plt.xlabel('airmass')
plt.ylabel('seeing')

In [ ]:
ccdvisit_subset = ccdVisitTable[['visitId', 'band', 'ra', 'dec','detector','zenithDistance','obsStartMJD']]

In [ ]:
ccdvisit_subset 

In [ ]:
# https://pipelines.lsst.io/modules/lsst.geom/getting-started.html
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()
func_zendtoAirmass = lambda x : 1./np.cos(func_degToRad(x))
#func_wrap = lambda x : Angle(x,lsst.geom.radians).wrap(180.*lsst.geom.degrees)

In [ ]:
ccdvisit_subset["zenithDistance_rad"]= ccdvisit_subset["zenithDistance"].apply(func_degToRad)
ccdvisit_subset["airmass"]= ccdvisit_subset["zenithDistance"].apply(func_zendtoAirmass)

In [ ]:
ccdvisit_subset

In [ ]:
#ccdvisit_subset_u = ccdvisit_subset[ccdvisit_subset.band == 'u']
ccdvisit_subset_g = ccdvisit_subset[ccdvisit_subset.band == 'g']
ccdvisit_subset_r = ccdvisit_subset[ccdvisit_subset.band == 'r']
ccdvisit_subset_i = ccdvisit_subset[ccdvisit_subset.band == 'i']
#ccdvisit_subset_z = ccdvisit_subset[ccdvisit_subset.band == 'z']
#ccdvisit_subset_y = ccdvisit_subset[ccdvisit_subset.band == 'y']
Ng = len(ccdvisit_subset_g)
Nr = len(ccdvisit_subset_r)
Ni = len(ccdvisit_subset_i)
print(Ng,Nr,Ni)

In [ ]:
sns.color_palette("Spectral", as_cmap=True)

In [ ]:
cmap_g = ListedColormap(sns.color_palette("Spectral", Ng))
cmap_r = ListedColormap(sns.color_palette("Spectral", Nr))
cmap_i = ListedColormap(sns.color_palette("Spectral", Ni))

#patch_to_color[patch] = cmap.colors[idx]

In [ ]:
all_colors_g = [cmap_g.colors[idx] for idx in range(Ng)]
all_colors_r = [cmap_r.colors[idx] for idx in range(Nr)]
all_colors_i = [cmap_i.colors[idx] for idx in range(Ni)]

In [ ]:
ra_g = ccdvisit_subset_g['ra'].apply(func_degToRad) 
dec_g = ccdvisit_subset_g['dec'].apply(func_degToRad) 

In [ ]:
# Galactic plane
gal_long = np.linspace(-180.,180,360)
gal_lat = np.zeros((360))
coordinates_galactic_planes = SkyCoord(l=gal_long*u.degree, b=gal_lat*u.degree, frame='galactic')
gp_radec = coordinates_galactic_planes.transform_to('icrs')  
# for galactic plane
# gp_radec.ra : 0., 360.
# gp_radec.dec : -90, 90
gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)
# gp_ra_toplot -180, 180
#wrapping angle : https://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4),dpi=200)

leg = ax.get_legend()
#ax.scatter(ccdvisit_subset_g["obsStartMJD"].values,ccdvisit_subset_g["ra"].values ,c=all_colors_g)
ccdvisit_subset_g.plot.scatter(x="obsStartMJD",y="ra",marker="o",c=all_colors_g,ax=ax,legend=leg,label="G")
ccdvisit_subset_r.plot.scatter(x="obsStartMJD",y="ra",marker="^",c=all_colors_r,ax=ax,legend=leg,label="R")
ccdvisit_subset_i.plot.scatter(x="obsStartMJD",y="ra",marker="v",c=all_colors_i,ax=ax,legend=leg,label="I")
ax.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4),dpi=200)
leg = ax.get_legend()
#ax.scatter(ccdvisit_subset_g["obsStartMJD"].values,ccdvisit_subset_g["ra"].values ,c=all_colors_g)
ccdvisit_subset_g.plot.scatter(x="obsStartMJD",y="dec",marker="o",c=all_colors_g,ax=ax,legend=leg,label="G")
ccdvisit_subset_r.plot.scatter(x="obsStartMJD",y="dec",marker="^",c=all_colors_r,ax=ax,legend=leg,label="R")
ccdvisit_subset_i.plot.scatter(x="obsStartMJD",y="dec",marker="v",c=all_colors_i,ax=ax,legend=leg,label="I")
ax.grid()

In [ ]:
dict_tract_to_coords_rad

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(12,6),dpi=200)
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="r",label="Galactic Plane",s=1)
ax.scatter(ra_g.values-np.pi,dec_g.values,marker='+',c=all_colors_g)
ax.legend()
ax.grid()

index=0
for key, value in dict_tract_to_coords_rad.items():
    tract_num_str= str(key)
    x = value["ra"] - np.pi
    y = value["dec"] 
    
    if index%2 == 0:
        dy= -Angle(3,lsst.geom.degrees).asRadians()
        dx= -Angle(3,lsst.geom.degrees).asRadians()
    else:
        dy= Angle(3,lsst.geom.degrees).asRadians()
        dx= Angle(3,lsst.geom.degrees).asRadians()
        
    ax.text(x+dx, y+dy, tract_num_str, fontsize=8,color="b")
    index+=1

plt.suptitle(instrument)

In [ ]:
# plot
# x arg in rad must be (-2pi,2pi), y arg in rad  must be in ( -pi,pi )
fig = plt.figure(figsize=(12,6),dpi=200)
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111)
ax.scatter(gp_ra_toplot.degree, gp_radec.dec.degree,c="r",label="Galactic Plane",s=1)
ax.scatter(ccdvisit_subset_g['ra'].values -180.,ccdvisit_subset_g['dec'].values,marker='+',c=all_colors_g)
ax.legend()
ax.grid()

index=0
for key, value in dict_tract_to_coords_andvisits.items():
    
    tract_num_str= str(key)
    x = value['ra_deg'].asDegrees() - 180
    y = value["dec_deg"].asDegrees() 
    
    if index%2 == 0:
        dy= -5
        dx= -5
    else:
        dy= 5
        dx= 5
        
    ax.text(x+dx, y+dy, tract_num_str, fontsize=8,color="b")
    index+=1

plt.suptitle(instrument)

## sourceTable_visit

In [ ]:
if 0:
    all_data = []
    data_product = 'sourceTable_visit'
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= "instrument='LSSTComCamSim'")
    for i, ref in enumerate(datasetRefs):
        if (i % 500 == 0):
            print(f"======================== datasetType = {data_product} ============================================")
            print("fullId..................:",ref.dataId.full)
        try:
            data = butler.get(data_product, dataId=ref.dataId )  
            all_data.append(data)
       
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)       

    df = pd.concat(all_data)
    filename = f"out-{data_product}.csv" 
    df.to_csv(filename)

## How do I access an object table?

In [ ]:
objectTable_refs = sorted(butler.registry.queryDatasets('objectTable_tract'))
N = len(objectTable_refs)
objectTable = butler.get(objectTable_refs[0])
objectTable[['coord_ra','coord_dec','tract','patch']]

In [ ]:
objectTable[['tract','patch']].groupby(["tract","patch"]).size()

In [ ]:
patches = objectTable["patch"].unique()

In [ ]:
patches

In [ ]:
from collections import OrderedDict
tracts_patchs_dict = OrderedDict()
objectTable_refs = sorted(butler.registry.queryDatasets('objectTable_tract'))
N = len(objectTable_refs)
for idx in range(N):
    objectTable = butler.get(objectTable_refs[idx])
    tract = objectTable["tract"].unique()[0] 
    patches = objectTable["patch"].unique()
    tracts_patchs_dict[tract] = patches 
    del objectTable

In [ ]:
tracts_patchs_dict

In [ ]:
import pickle
# Store data (serialize)
filetractpatch_out = 'lsstcomcam_tractspatchs_20250219.pickle'
with open(filetractpatch_out, 'wb') as handle:
    pickle.dump(tracts_patchs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## preSourceTable

In [ ]:
if 0:
    all_data = []
    data_product = 'preSourceTable'
    datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= "instrument='LSSTComCamSim'")
    for i, ref in enumerate(datasetRefs):
        if (i % 500 == 0):
            print(f"======================== datasetType = {data_product} ============================================")
            print("fullId..................:",ref.dataId.full)
        try:
            data = butler.get(data_product, dataId=ref.dataId )  
            all_data.append(data)
       
        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)       

    df = pd.concat(all_data)
    filename = f"out-{data_product}.csv" 
    df.to_csv(filename)